In [4]:
import pandas as pd
import sklearn
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.metrics import mean_squared_error, root_mean_squared_error
import statistics

### Data load and prep

In [17]:
df_train = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet')

### Q1

In [19]:
len(df_train.columns)

19

In [6]:
df_val = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet')

In [5]:
df_train['lpep_dropoff_datetime'] = pd.to_datetime(df_train['tpep_dropoff_datetime'], utc = True)
df_train['lpep_pickup_datetime'] = pd.to_datetime(df_train['tpep_pickup_datetime'], utc = True)
df_train['duration'] = df_train.lpep_dropoff_datetime - df_train.lpep_pickup_datetime
df_train['duration'] = df_train['duration'].apply(lambda x: x.total_seconds()/60)

In [7]:
df_val['lpep_dropoff_datetime'] = pd.to_datetime(df_val['tpep_dropoff_datetime'], utc = True)
df_val['lpep_pickup_datetime'] = pd.to_datetime(df_val['tpep_pickup_datetime'], utc = True)
df_val['duration'] = df_val.lpep_dropoff_datetime - df_val.lpep_pickup_datetime
df_val['duration'] = df_val['duration'].apply(lambda x: x.total_seconds()/60)

### Q2

In [9]:
statistics.stdev(df_train.duration)

42.59435124195457

### Q3

In [11]:
len(df_train[(df_train.duration >= 1) & (df_train.duration <= 60)])/len(df_train)

0.9812202822125979

### Q4

In [12]:
df_train = df_train[(df_train.duration >= 1) & (df_train.duration <= 60)]

In [13]:
df_val = df_val[(df_val.duration >= 1) & (df_val.duration <= 60)]

In [14]:
categorical = ['PULocationID', 'DOLocationID']

df_train[categorical] = df_train[categorical].astype(str)
df_val[categorical] = df_val[categorical].astype(str)

In [15]:
train_dicts = df_train[categorical].to_dict(orient = 'records')

dv = DictVectorizer() # converts list of dictionaries to a matrix suitable for ML algorithms, numerical features are left as is and categorical features are one-hot encoded
x_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical].to_dict(orient = 'records')
x_val = dv.transform(val_dicts)

In [20]:
x_val

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 5711894 stored elements and shape (2855951, 515)>

### Q5

In [27]:
target = 'duration'

y = df[target].values
y_val = df_val[target].values

In [29]:
lr = LinearRegression()
lr.fit(x, y)
y_pred = lr.predict(x)
root_mean_squared_error(y, y_pred)

7.6492624397080675

### Q6

In [30]:
y_pred = lr.predict(x_val)
root_mean_squared_error(y_val, y_pred)

7.81181211389241